# Cleaning of the raw csv file for processing

In [93]:
import pandas as pd

# Load the CSV file
df = pd.read_csv("job_postings_raw.csv")


# Dropping rows with missing critical data
df.dropna(subset=['job_id', 'company_id'], inplace=True)
df.dropna(subset=["med_salary", "max_salary", "min_salary"], how="all", inplace=True)

# Dropping unnecessary columns
df.drop(['job_posting_url', 'description', 'application_url', 'posting_domain', 'sponsored', 
         'compensation_type', 'scraped', 'closed_time', 'skills_desc'], axis=1, inplace=True)


def convert_to_yearly(row):
    salary = row["med_salary"] if pd.notna(row["med_salary"]) else (row["max_salary"] + row["min_salary"]) / 2
    pay_period = row["pay_period"]
    work_type = row["formatted_work_type"]
    
    if pay_period == "HOURLY":
        if work_type == "Full-time":
            return salary * 8 * 5 * 50  # 8 hours/day, 5 days/week, 50 weeks/year
        elif work_type == "Part-time":
            return salary * 5 * 5 * 50  # 5 hours/day, 5 days/week, 50 weeks/year
        else:
            return salary * 8 * 5 * 50  

    elif pay_period == "WEEKLY":
        return salary * 52  # Default weekly conversion

    elif pay_period == "MONTHLY":
        return salary * 12  # Standard conversion for monthly salaries

    elif pay_period == "ONCE":
        return salary  # One-time payment, no conversion

    elif pay_period == "YEARLY":
        return salary  # Already in yearly format

    return None  # If no valid pay period

# Apply conversion function to the DataFrame
df["yearly_med_salary"] = df.apply(convert_to_yearly, axis=1)
df = df[df['yearly_med_salary'].notna()]
df.drop(['max_salary', 'med_salary', 'min_salary'], axis=1, inplace=True)
df["pay_period"] = "YEARLY"

# Filling missing data for non-critical columns
df['formatted_experience_level'] = df['formatted_experience_level'].fillna("Not Mentioned")
df['remote_allowed'] = df['remote_allowed'].fillna(0)
df['views'] = df['views'].fillna(0)
df['applies'] = df['applies'].fillna(0)

# Converting Unix timestamps to datetime format
df['original_listed_time'] = pd.to_datetime(df['original_listed_time'], unit='ms')
df['listed_time'] = pd.to_datetime(df['listed_time'], unit='ms')
df['expiry'] = pd.to_datetime(df['expiry'], unit='ms')


#Handling of outliers
Q1 = df['yearly_med_salary'].quantile(0.25)
Q3 = df['yearly_med_salary'].quantile(0.75)
IQR = Q3 - Q1
df = df[(df['yearly_med_salary'] >= Q1 - 1.5 * IQR) & (df['yearly_med_salary'] <= Q3 + 1.5 * IQR)]

# Save the processed data
df.to_csv("job_posting_clean.csv", index=False)